In [15]:
import sys
import cv2, os
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
from imgaug import parameters as iap

In [1]:
def load_image(image_file):
    """
    Load RGB images from a file.
    """
    return mpimg.imread('../../' + os.path.join(image_file.strip()))

In [17]:
def show_image(image):
    """
    Show image.
    """
    plt.imshow(image)

In [18]:
def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    """
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

In [19]:
def random_contrast(image):
    """
    Randomly add contrast to the image.
    """
    contrast = iaa.GammaContrast((0.1, 1.5))
    return contrast.augment_image(image)

In [20]:
def random_sharpen(image):
    """
    Randomly add shapen to the image.
    """
    sharpen = iaa.Sharpen(alpha=(0.1, 0.5))
    return sharpen.augment_image(image)

In [21]:
def random_addition_augment(image):
    """
    Randomly choose addition augment to perform.
    - Salt and Pepper Noise
    - Gaussian Blur
    - 
    """
    aug = iaa.OneOf([
        iaa.SaltAndPepper(p=(0.01, 0.03)),
        iaa.GaussianBlur(iap.Uniform(0.1, 2.0)),
        iaa.Dropout(p=(0.01, 0.1)),
        iaa.Noop()
    ])
    return aug.augment_image(image)

In [22]:
def rgb2yuv(image):
    """
    Convert the image from RGB to YUV (This is what the NVIDIA model does)
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

In [23]:
def resize(image):
    """
    Resize the image to the input shape used by the network model
    """
    IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)

In [24]:
def preprocess(image):
    """
    Combine all preprocess functions into one
    """
    image = resize(image)
    image = rgb2yuv(image)
    return image

In [25]:
def augument(image):
    """
    We perform augumentation. Transformations include change in contrast, and brightness,
    as well as addition of Gaussian blur, Gaussian noise, salt-and pepper noise and region dropout.
    """
    image = load_image(image)
    image = random_contrast(image)
    image = random_brightness(image)
    image = random_sharpen(image)
    image = random_addition_augment(image)
    return image

In [38]:
def batch_generator(inputs_data, controls_data, batch_size, is_training):
    """
    Generate training image give inputs data and associated controls
    """
    IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
    CONTROL_ARGUMENTS = 3 # throttle, steer, brake
    inputs_image = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    inputs_speed = np.empty([batch_size])
    inputs_cmd = np.empty([batch_size])
    controls = np.empty([batch_size, CONTROL_ARGUMENTS])
    count = 0
    while True:
        count = 1
        i = 0
        for index_folder in np.random.permutation(4):
            for index in np.random.permutation(inputs_data[index_folder].shape[0]):
                image, speed, cmd = inputs_data[index_folder][index]
                throttle, steer, brake = controls_data[index_folder][index]
                throttle, steer, brake = float(throttle), float(steer), float(brake)
                speed, cmd = float(speed)/100, int(cmd)/10
                # argumentation
                try:
                    if is_training and np.random.rand() < 0.6:
                        image = augument(image)
                    else:
                        image = load_image(image) 
                except:
                    continue
                # add the image and other inputs and controls to the batch

                # scale input
                image = preprocess(image)
                image = image.astype(np.float32)
                image = np.multiply(image, 1.0 / 255.0)
                inputs_image[i] = image
                inputs_speed[i] = speed
                inputs_cmd[i] = cmd
                controls[i] = [throttle*0.25, steer*0.25, brake*0.25]
                i += 1

                if i == batch_size/4 * count:
                    count += 1
                    break
        yield [inputs_image, inputs_speed.reshape(-1,1), inputs_cmd.reshape(-1,1)], controls